In [1]:
import numpy as np

### training process

- loss_mask
    - 指示哪些token的预测应该被计入损失（例如，在多轮对话中，可能只计算回答部分的损失，而不计算提示部分的损失）。它通常排除了最后一个token，因为最后一个token没有下一个token可供预测。
    - multiturn
        - mask prompts，只 supervised response 部分，即只计算 response 的 loss
- logits
    - shape: `[batch_size, seq_len, vocab_size]`
- 对齐 logits 与 labels
    - `labels` (目标) 通常是 `input_ids` 向左移动一位得到 (`input_ids[:, 1:]`)。
    - `shift_logits` 是 logits 去掉最后一个时间步的预测 (`logits[..., :-1, :]`)，以与 labels 对齐。
- 逐 token 计算 loss
    - `nn.CrossEntropyLoss(reduction="none")`

### data & prompt

- `sft_dataset.py`: SFTDataset
    - eos_token: `<|im_end|>`
        - im: instruct message（from base model to instruct model）
        - 注意不是 `<|endoftext|>`(`pad_token`)，
```python
 # apply chat template
prompt_chat = [{"role": "user", "content": prompt}]

# string
prompt_chat_str = tokenizer.apply_chat_template(prompt_chat, add_generation_prompt=True, tokenize=False)
response_chat_str = response + tokenizer.eos_token
```

```
<|im_start|>system
You are a helpful assistant.
<|im_end|>
<|im_start|>user
你好，请问今天天气怎么样？
<|im_end|>
<|im_start|>assistant
你好！请告诉我您所在的城市，我可以为您查询天气。
<|im_end|>
```

### traing loss

$$
\text{CE}=-\log(P_{\text{true\_token}})
$$

- 假如训练到 0.3 的 loss 水平
    - $P_{\text{true\_token}}=\exp(-0.3)=0.7408$
- 考虑到 qwen vocab size 152064 的水平
    - 瞎猜 $\text{CE}_{\text{rand}}=-\log(\frac1{152064})=11.93$
- 从 PPL（perplexity）的角度
    - $PPL=\exp(\text{CE})$
        - CEL = 0.3 => PPL = 1.35 (能将下一个最可能的词的范围缩小到好像平均只有 1 到 2 个（1.3634）选项一样。)

In [5]:
np.exp(-0.3), np.log(152064), np.exp(0.3), np.exp(11.93)

(0.7408182206817179,
 11.932056763842207,
 1.3498588075760032,
 151751.56167916086)

### overfitting

- 需要持续监控 training losses & val losses
    - training losses 不断地下降，但 val losses 先下降后上升；
    - 我这边的经验即是，2个epochs，就会达到 val losses 较低的水平，后续会上升；